In [ ]:
# imports:
import numpy as np
import matplotlib.pyplot as plt
import re
import geopandas as gpd
import os
import pandas as pd
import rioxarray
import s3fs

## Intro

This notebook contains code for reading ATL08 data for Grand Mesa from OpenAltimetry.

In [ ]:
%matplotlib widget

## Import the Sentinel basemap

In [ ]:
# GDAL environment variables to efficiently read remote data
os.environ['GDAL_DISABLE_READDIR_ON_OPEN']='EMPTY_DIR' 
os.environ['AWS_NO_SIGN_REQUEST']='YES' 

# SAR Data are stored in a public S3 Bucket
url = 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/12/S/YJ/2016/S1B_20161121_12SYJ_ASC/Gamma0_VV.tif'

# These Cloud-Optimized-Geotiff (COG) files have 'overviews', low-resolution copies for quick visualization
XR=[725000.0, 767000.0]
YR=[4.30e6, 4.34e6]
# open the dataset
da = rioxarray.open_rasterio(url, overview_level=1).squeeze('band')#.clip_box([712410.0, 4295090.0, 797010.0, 4344370.0])
da=da.where((da.x>XR[0]) & (da.x < XR[1]), drop=True)
da=da.where((da.y>YR[0]) & (da.y < YR[1]), drop=True)
dx=da.x[1]-da.x[0]
SAR_extent=[da.x[0]-dx/2, da.x[-1]+dx/2, np.min(da.y)-dx/2, np.max(da.y)+dx/2]

# Prepare coordinate transformations into the basemap coordinate system
from pyproj import Transformer, CRS
crs=CRS.from_wkt(da['spatial_ref'].spatial_ref.crs_wkt)
to_image_crs=Transformer.from_crs(crs.geodetic_crs, crs)
to_geo_crs=Transformer.from_crs(crs, crs.geodetic_crs)

corners_lon, corners_lat=to_geo_crs.transform(np.array(XR)[[0, 1, 1, 0, 0]], np.array(YR)[[0, 0, 1, 1, 0]])
lonlims=[np.min(corners_lat), np.max(corners_lat)]
latlims=[np.min(corners_lon), np.max(corners_lon)]

## Use IcePyx to query CMR for available granules

(Can probably do this with openAltimetry too)

In [ ]:
import requests
import icepyx as ipx

region_a = ipx.Query('ATL08', [lonlims[0], latlims[0], lonlims[1], latlims[1]], ['2018-12-01','2021-06-01'], \
                          start_time='00:00:00', end_time='23:59:59')

In [ ]:
region_a.avail_granules()

In [ ]:
ATLAS_re=re.compile('ATL.._(?P<year>\d\d\d\d)(?P<month>\d\d)(?P<day>\d\d)\d+_(?P<track>\d\d\d\d)')

date_track=[]
for count, item in enumerate(region_a.granules.avail):
    granule_info=ATLAS_re.search(item['producer_granule_id']).groupdict()
    date_track += [ ('-'.join([granule_info[key] for key in ['year', 'month', 'day']]), granule_info['track'])]

## Define a function to read ATL08 from OpenAltimetry

This function will go in its own standalone Python file

In [ ]:
def get_OA_ATL08(date_track, lonlims, latlims, beamnames=["gt1l","gt1r","gt2l","gt2r","gt3l","gt3r"]):
    '''
    retrieve ICESat2 ATL03 data from openAltimetry
    
    Inputs:
        date_track: a list of tuples.  Each contains a date string "YYYY-MM-DD" and track number (4-character string)
        lonlims: longitude limits for the search
        latlims: latitude limits for the search
        beamnames: list of strings for the beams
    outputs:
        a dict containing ATL03 data by beam name
    
    Due credit:
        Much of this code was borrowed Philipp Arndt's Pond Picker repo: https://github.com/fliphilipp/pondpicking
    '''
      
    IS2_data={}
    for this_dt in date_track:
        this_IS2_data={}
        for beamname in beamnames:
            oa_url = 'https://openaltimetry.org/data/api/icesat2/atl08?minx={minx}&miny={miny}&maxx={maxx}&maxy={maxy}&trackId={trackid}&beamName={beamname}&outputFormat=json&date={date}&client=jupyter'
            oa_url = oa_url.format(minx=lonlims[0],miny=latlims[0],maxx=lonlims[1], maxy=latlims[1], 
                                   trackid=this_dt[1], beamname=beamname, date=this_dt[0], sampling='true')
            #.conf_ph = ['Noise','Buffer', 'Low', 'Medium', 'High']
            if True:
                r = requests.get(oa_url)
                data = r.json()
                D={}
                D['lat_seg'] = []
                D['lon_seg'] = []
                D['h_ground'] = []
                D['h_canopy']=[]
                for series in data['series']:
                    for p in series['lat_lon_elev_canopy']:
                        D['lat_seg'].append(p[0])
                        D['lon_seg'].append(p[1])
                        if p[2] is not None:
                            D['h_ground'].append(p[2])
                        else:
                            D['h_ground'].append(np.NaN)
                        if p[3] is not None:
                            D['h_canopy'].append(p[3])
                        else:
                            D['h_canopy'].append(np.NaN)
                D['x_seg'], D['y_seg']=to_image_crs.transform(D['lat_seg'], D['lon_seg'])
                for key in D:
                    D[key]=np.array(D[key])
                if len(D['lat_seg']) > 0:
                    this_IS2_data[beamname]=D
            #except Exception as e:
            #    print(e)
            #    pass
        if len(this_IS2_data.keys()) > 0:
            IS2_data[this_dt] = this_IS2_data
    return IS2_data

## Get the data

In [ ]:
D08 = get_OA_ATL08(date_track, lonlims, latlims)

In [ ]:
plt.figure()
plt.imshow(np.array(da)[::-1,:], origin='lower', extent=SAR_extent, cmap='gray', clim=[0, 0.5])#plt.figure();

for dt, day_data in D08.items():
    for beam, D in day_data.items():
        plt.plot(D['x_seg'], D['y_seg'], '.', markersize=3, label=str(dt))

